In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('result/original_data.csv')

In [3]:
data['gender']=data['gender'].map(lambda x:1 if x=='M' else 0)
divide=data['height_first']/100
data['BMI']=data['weight_first']/(divide*divide)
data.drop(['weight_first','height_first'], axis=1, inplace=True)
for i in ['WHITE','WHITE - RUSSIAN','WHITE - BRAZILIAN','WHITE - OTHER EUROPEAN','WHITE - EASTERN EUROPEAN']:
    data.loc[data[data['ethnicity']==i].index, 'ethnicity']=0
for i in ['ASIAN','ASIAN - CHINESE','ASIAN - ASIAN INDIAN','ASIAN - THAI','ASIAN - OTHER', 'ASIAN - VIETNAMESE',
         'ASIAN - FILIPINO', 'ASIAN - JAPANESE','ASIAN - KOREAN','ASIAN - CAMBODIAN']:
    data.loc[data[data['ethnicity']==i].index, 'ethnicity']=1 
for i in ['BLACK/AFRICAN AMERICAN','BLACK/CAPE VERDEAN',
       'BLACK/HAITIAN','BLACK/AFRICAN',]:
    data.loc[data[data['ethnicity']==i].index, 'ethnicity']=2
data['ethnicity']=data['ethnicity'].map(lambda x:3 if type(x)!=int else x)
data.drop(['IBIL.csv'],axis=1,inplace=True)

In [4]:
data['365_surviving_time']=(data['dod_h']-data['admittime_h'])/24
data['365_surviving_time']=data['365_surviving_time'].fillna(366)
data.loc[data[data['365_surviving_time']>=365].index, '365_surviving_time']=365

data['30_surviving_time']=(data['dod_h']-data['admittime_h'])/24
data['30_surviving_time']=data['30_surviving_time'].fillna(31)
data.loc[data[data['30_surviving_time']>=30].index, '30_surviving_time']=30

In [5]:
def quantized(dataframe,variables_list,number):
    dataframe=dataframe.copy()
    for i in variables_list:
        point=[]
        for num in range(number+1):
            point.append(dataframe[i].quantile(0+(1/number)*(num)))
        print(i,':',point)          
        for j in range(1,number+1):
            columns_index=i+str(round(point[j-1],2))+'-'+str(round(point[j],2))
            dataframe[columns_index]=0
            indexes=dataframe[(dataframe[i]<=point[j]) & (dataframe[i]>=point[j-1])].index
            dataframe.loc[indexes, columns_index]=1
    return dataframe
data=quantized(data,['PTT.csv','PT.csv','INR.csv'],4)
data=quantized(data,['age','sofa','sapsii'],2)

PTT.csv : [16.9, 26.8, 31.3, 39.6, 150.0]
PT.csv : [9.2, 13.2, 14.3, 16.3, 150.0]
INR.csv : [0.0, 1.1, 1.3, 1.6, 22.8]
age : [20.356943556570265, 69.46984113394217, 301.27569536402837]
sofa : [0.0, 4.0, 20.0]
sapsii : [6.0, 38.0, 114.0]


In [6]:
data['sirs_dummy']=data['sirs'].map(lambda x:'sirs='+str(x))
data=data.join(pd.get_dummies(data['sirs_dummy']))

In [7]:
data.to_csv('result/original_dataV2.csv',index=False)